# pgmpy를 사용한 베이즈 정리 적용
### dss pg 433~435
~~~
pgmpy 패키지는 베이즈 정리에 적용하는 BayesianModel 클래스를 제공한다. 
베이즈 정리를 적용하려면 조건부확률을 구현하는 TabularCPD 클래스를 사용하여 사전확률과 가능도를 구현해야 한다.
~~~
**TabularCPD**
* variable: 확률변수의 이름 문자열
* variable_card: 확률변수가 가질 수 있는 경우의 수
* value: 조건부확률 배열. 하나의 열(column)이 동일 조건을 뜻하므로 하나의 열의 확률 합은 1이여야 한다.
* evidence: 조건이 되는 확률변수의 이름 문자열 list
* evidence_card: 조건이 되는 확률변수가 가질 수 있는 경우의 수의 list

~~~
TabularCPD는 원래 조건부확률을 구현하기 위한 것이지만 evidence=None, evidence_card=None으로 인수를 주면 일반적인 확률도 구현할 수 있다.
~~~

### #인문학적인 내용 설명
~~~
제약회사에서 환자가 특정한 병에 걸린지 확인하는 시약 생산.
그 약 test 결과 99% 확률로 양성 반응을 보임.
검사 결과가 양성 반응이 나왔을 때 해당 환자가 실제로 병에 걸렸을 확률은?
~~~

### 1. 확률변수 X를 이용하여 아래 내용들 정의
~~~ 
병에 걸렸을 사전확률: P(D)=P(X=1)
병에 걸리지 않았을 사전확률: P(D 여사건)=P(X=0)
~~~

In [6]:
from pgmpy.factors.discrete import TabularCPD
import numpy as np

In [7]:
cpd_X=TabularCPD('X', 2, [[1-0.002], [0.002]])
print(cpd_X)

+------+-------+
| X(0) | 0.998 |
+------+-------+
| X(1) | 0.002 |
+------+-------+


### #인문학적인 내용 설명
* X(0): 전체에서 병에 걸리지 않은 사람의 비율
* X(1): 전체에서 병에 걸린 사람의 비율 (걸린 사람이 0.2%인 희귀병이다: X(1)=0.002)

### 2. 조건부확률 P(Y|X)를 구현
~~~
양성 반응이 나올 확률: P(S)=P(Y=1)
음성 반응이 나올 확률: P(S 여사건)=P(Y=0)
위의 내용을 나타내는 확률변수 Y를 정의한다.
~~~
**확률변수 Y의 확률을 베이지 묘형에 넣을 때는 TabularCPD를 사용한 조검부 확률(P(Y|X)) 형태로 넣어야 한다.**
~~~
아래 코드로 조건부확률 P(Y|X)를 구현
~~~

In [9]:
cpd_Y_on_X=TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]), evidence=['X'], evidence_card=[2])
print(cpd_Y_on_X)

+------+------+------+
| X    | X(0) | X(1) |
+------+------+------+
| Y(0) | 0.95 | 0.01 |
+------+------+------+
| Y(1) | 0.05 | 0.99 |
+------+------+------+


### #인문학적인 내용 설명
* X(0)& Y(1)= 0.05: 병에 걸리지 않은 사람에게 시약 검사 시 양성반은 (False Positive)가 나타날 확률
* X(1)& Y(1)= 0.99: 병에 걸린 사람이 양성이 나올 확률
~~~
Y(1)은 양성이 나올 경우를 말하는 것이고 X(1)은 병에 걸릴 확률을 의미한다. 
TabularCPD('Y', 2, np.array([[0.95, 0.01], [0.05, 0.99]]), evidence=['X'], evidence_card=[2])
위의 코드는 X의 지정 값이 참일 때 Y의 지정 결과가 나올 확률을 찾는 함수이다. P(Y|X)
각각 array를 Y(0)일 때를 한 list로 묶고 Y(1)을 다른 list로 묶는다
~~~

### 3. 확률 모형인 BayesianModel 클래스 객체를 만들기
~~~
확률변수들이 어떻게 결합되어 있는지 나타내는 확률 모형
~~~
**BayesianModel (variable)**

* variables: 확률 모형이 포함하는 확률변수 이름 문자열의 list

**BayesianModel은 다음과 같은 메서드를 지원한다.**

* add_cpds(): 조건부확률 푸가
* check_model(): 모형이 정상적인지 확인 True면 정상

In [10]:
from pgmpy.models import BayesianModel

In [20]:
model1=BayesianModel([('X', 'Y')])
model1.add_cpds(cpd_X, cpd_Y_on_X)
model1.check_model()

True

### 4. VariableElimination(변수 제거법)을 사용한 추정
~~~
BayesianModel는 변수 제거법을 사용한 추정을 제공
1. VariableElimination 클래스로 추정 객체 (reference)를 만든다.
2. query() 메서드를 사용하면 사후확률을 계산
~~~
**query (variables, evidences)**
* variables: 사후확률을 계산할 확률변수 이름 list
* evidences: 조건이 되는 확률변수의 값을 나타내는 dictionary
~~~
여기에서는 pgmpy 패키지를 이용하여 베이즈 정리를 적용할 수 있다는 것만 알면 된다.
자세한 내용은 추후 확률적 그래프 모형 (Probabilistic Graphical Model)에서 다룬다
~~~

In [21]:
from pgmpy.inference import VariableElimination

In [22]:
inference=VariableElimination(model1)
posterior=inference.query(['X'], evidence={'Y': 1})
print(posterior)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(0) |   0.9618 |
+------+----------+
| X(1) |   0.0382 |
+------+----------+


### #인문학적인 내용 설명
~~~
X(1)=0.0382은 양성 반응이 나왔을 때 병에 결릴 확률 (Y(1)이 참이라는 가정 하에 X(1)일 확률)을 의미한다
X(0)=0.9618은 양성 반응이 나왔을 때 병에 안 결릴 확률
~~~

In [19]:
print(posterior['Y'])

TypeError: 'DiscreteFactor' object is not subscriptable